In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from numpy import sqrt

from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

# Load data

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/recomment_system/Do_an/Collaborative_Filtering/Data/data-collaborative-fittering/final_reviews_train_than30.csv")
test = pd.read_csv("/content/drive/MyDrive/recomment_system/Do_an/Collaborative_Filtering/Data/data-collaborative-fittering/final_reviews_test_than30.csv")

book_information = pd.read_csv("/content/drive/MyDrive/recomment_system/Do_an/Collaborative_Filtering/Data/data-collaborative-fittering/final_books_than30.csv")

In [ ]:
train.head()

,id,product_id,user_id,user_name,rating,content,thank_count,rating_average,count
0,6081499,3734909.0,88616.0,Trương Minh Đăng,5,NaN,0,4.8,1
1,12731297,14961285.0,16848626.0,Phạm Văn Tùng,5,NaN,0,4.7,1
2,7849857,68774130.0,8046945.0,Minh Giải,5,NaN,0,5.0,1
3,15666018,152449148.0,1333575.0,Genny Lee,5,bộ truyện rất hay và giàu tính nhân văn.,0,4.9,1
4,17241576,655679.0,24504222.0,Nhật Việt,5,NaN,0,4.9,1


In [ ]:
train.shape

(140839, 9)

In [ ]:
test.shape

(1423, 9)

In [ ]:
book_information.shape

(16587, 17)

In [ ]:
book_information.head(3)

,category,url,id,sku,name,price,rating_average,review_count,all_time_quantity_sold,description,specifications,seller_id,seller_sku,seller_name,seller_store_id,count_specification,attributes
0,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...,https://tiki.vn/product-p141483251.html?spid=1...,141483251.0,8.070578e+12,Sách: Thai Giáo Theo Chuyên Gia - 280 Ngày Mỗi...,80500.0,5.0,58.0,508.0,"\nThai Giáo Theo Chuyên Gia, 280 Ngày, Mỗi Ngà...","{""name"": ""Th\u00f4ng tin chung"", ""attributes"":...",36846.0,3.544877e+12,Minh Long Book,36125.0,5,"{'id': 141483251.0, 'publisher_vn': 'Minh Long..."
1,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...,https://tiki.vn/product-p125362334.html?spid=1...,125362334.0,8.806987e+12,Dinh Dưỡng Thai Kỳ (Bà Bầu Ăn Gì Giúp Con Phát...,103950.0,4.8,41.0,486.0,“Dinh dưỡng thai kỳ” giống như một cuốn bách k...,"{""name"": ""Th\u00f4ng tin chung"", ""attributes"":...",1.0,6.946825e+12,Tiki Trading,40395.0,7,"{'id': 125362334.0, 'publisher_vn': 'Booklife'..."
2,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...,https://tiki.vn/product-p58213080.html?spid=58...,58213080.0,4.467229e+12,Montessori – Phương Pháp Giáo Dục Toàn Diện Ch...,95200.0,5.0,16.0,116.0,MONTESSORI – PHƯƠNG PHÁP GIÁO DỤC TOÀN DIỆN CH...,"{""name"": ""Th\u00f4ng tin chung"", ""attributes"":...",53660.0,1.012848e+12,Nhà sách Fahasa,51001.0,6,"{'id': 58213080.0, 'publisher_vn': '1980 Books..."


# Một vài hàm lấy dữ liệu

In [ ]:
def get_rating(userid, bookid):
    return (train.loc[((train.user_id== userid)&(train.product_id== bookid)), 'rating'].iloc[0])
#------------------------
def get_bookid(userid): 
    return (train.loc[(train.user_id==userid), 'product_id'].tolist())
#----------------------
def get_userid(bookid): 
    return (train.loc[(train.product_id==bookid), 'user_id'].tolist())
#------------------------
def get_food_title(bookid):
    return (book_information.loc[(book_information.id==bookid), 'name'].iloc[0])
#------------------------
def get_average_rating(userid):
    rating_user = train.loc[((train.user_id== userid)), 'rating']
    return np.mean(rating_user)
#-----------------------
def get_average_rating_book(bookid):
    rating_book = train.loc[((train.product_id== bookid)), 'rating']
    return np.mean(rating_book)

In [ ]:
get_average_rating_book(58543601.0)

4.826086956521739

In [ ]:
average_rating_userid = {}
for i in train["user_id"].unique():
    average_rating_userid[i]=get_average_rating(i)

# Các hàm tính độ tương tự 

## Độ đo Cosine

In [ ]:
def cosine_similarity_item(book1, book2):
    both_watch_count = 0
    for userid in train.loc[train.product_id == book1, 'user_id'].to_list():
        for userid in train.loc[train.product_id == book2, 'user_id'].tolist():
            both_watch_count +=1
    if both_watch_count == 0:
        return 0

    rating1 = []
    rating2 = []

    for userid in train.loc[train.product_id == book1, 'user_id'].to_list():
        if userid in train.loc[train.product_id == book2, 'user_id'].tolist():
            rating1.append(get_rating(userid, book1))
            rating2.append(get_rating(userid, book2))

    if (norm(rating1)*norm(rating2)) == 0:
        return 0
    return dot(rating1, rating2)/(norm(rating1)*norm(rating2))

In [ ]:
cosine_similarity_item(10722462.0, 58543601.0)

0

## Độ đo tương tự Pearson

In [ ]:
def pearson_correlation_item(book1, book2):
    both_watch_count = []
    for userid in train.loc[(train.product_id == book1), 'user_id'].tolist():
        if userid in train.loc[(train.product_id == book2), 'user_id'].tolist():
            both_watch_count.append(userid)
    if len(both_watch_count) == 0:
        return 0
    rating_sum_1 = sum([get_rating(userid, book1) for userid in both_watch_count])
    avg_rating_sum_1 = rating_sum_1/len(both_watch_count)

    rating_sum_2 = sum([get_rating(userid, book2) for userid in both_watch_count])
    avg_rating_sum_2 = rating_sum_2/len(both_watch_count)

    #Gia tri tren tu
    numerator = sum([(get_rating(userid, book1)- avg_rating_sum_1)*(get_rating(userid, book2) - avg_rating_sum_2) for userid in both_watch_count])
    denominator = sqrt(sum([pow((get_rating(userid, book1) - avg_rating_sum_1), 2) for userid in both_watch_count]))*sqrt(sum([pow((get_rating(userid, book2) - avg_rating_sum_2), 2) for userid in both_watch_count]))
    if denominator == 0:
        return 0
    return numerator/denominator

In [ ]:
pearson_correlation_item(10722462.0, 58543601.0)

0

# Hàm tính top k sách có độ tương đồng 

In [ ]:
def get_similarity_item(book1, number_of_book, metric_similar):
    book_ids = train.product_id.unique().tolist()

    similarity_score = []
    for foodID in book_ids[:500]:
        if (foodID != book1):
            if (metric_similar == "cosine"):
                similarity_score.append((cosine_similarity_item(book1, foodID), foodID))
            elif (metric_similar == "pearson"):
                similarity_score.append((pearson_correlation_item(book1, foodID), foodID))
    similarity_score.sort()
    similarity_score.reverse()
    return similarity_score[:number_of_book]

In [ ]:
get_similarity_item( 58543601.0, 10, "pearson")

[(1.0000000000000002, 54120846.0),
 (1.0000000000000002, 52789367.0),
 (1.0, 37875476.0),
 (1.0, 27582931.0),
 (0.9999999999999998, 57180817.0),
 (0.9999999999999998, 52329763.0),
 (0.6123724356957948, 52788072.0),
 (0.5000000000000001, 8885999.0),
 (0.5000000000000001, 3071853.0),
 (0.5, 70016692.0)]

# Hàm khuyến nghị sách

## Khuyến nghị dựa trên trung bình đánh giá

In [ ]:
def get_recommendation_tbdg_item(bookid, number_of_book, metric_similar):
    book_ids = train.product_id.unique().tolist()
    total = {}
    number_rating = {}
    List_similar = get_similarity_item(bookid, number_of_book, metric_similar)
    for peason, book in List_similar:
        for userid in get_userid(book):
            if userid not in get_userid(bookid) or get_rating(userid, bookid) == 0:
                if userid not in total:
                    total[userid] = 0
                    number_rating[userid] = 0
                    total[userid] += get_rating(userid, book)
                    if get_rating(userid, book) != 0:
                        number_rating[userid] +=1
                else:
                    total[userid] += get_rating(userid, book)
                    if get_rating(userid, book) != 0:
                        number_rating[userid] +=1
                
    ranking = [(to/number_rating[userid], userid) for userid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(userid, score) for score, userid in ranking]

    return recomendations

## Khuyến nghị dựa theo trung bình đánh giá có trọng số

In [ ]:
def get_recommendation_tbdgcts_item(bookid, number_of_book, metric_similar):
    book_ids = train.product_id.unique().tolist()
    total = {}
    similarity_sum = {}
    List_similar = get_similarity_item(bookid, number_of_book, metric_similar)
    for peason, book in List_similar:
        score = peason
        for userid in get_userid(book):
            if userid not in get_userid(bookid) or get_rating(userid, bookid) == 0:
                if userid not in total:
                    total[userid] = 0
                    total[userid] += get_rating(userid, book)*score
                    similarity_sum [userid] = 0
                    similarity_sum [userid] +=score
                else:
                    total[userid] += get_rating(userid, book)*score
                    similarity_sum [userid] +=score
                
    ranking = [(to/similarity_sum[userid], userid) for userid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(userid, score) for score, userid in ranking]

    return recomendations

## Khuyến nghị dựa trên khoảng đánh giá

In [ ]:
def get_recommendation_kdg_item(bookid, number_of_book, metric_similar):
    total = {}
    similarity_sum = {}
    List_similar = get_similarity_item(bookid, number_of_book, metric_similar)
    for peason, book in List_similar:
        score = peason
        for userid in get_userid(book):
            if userid not in get_userid(bookid) or get_rating(userid, bookid) == 0:
                if userid not in total:
                    total[userid] = 0
                    total[userid] += (get_rating(userid, book) - get_average_rating_book(book))*score
                    similarity_sum [userid] = 0
                    similarity_sum [userid] +=score
                else:
                    total[userid] += (get_rating(userid, book) - get_average_rating_book(book))*score
                    similarity_sum [userid] +=score
                
    ranking = [((get_average_rating_book(bookid) + to/similarity_sum[userid]), userid) for userid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(userid, score) for score, userid in ranking]

    return recomendations

## Khuyến nghị sách

In [ ]:
def get_recommendation_item(bookid, number_of_book, metrics_rating, metric_similar):
    if bookid not in list(train["product_id"].unique()):
        ranking = [(to, bookid) for bookid, to in average_rating_userid.items()]
        ranking.sort()
        ranking.reverse()
        recomendations = [(bookid, score) for score, bookid in ranking]
        return recomendations
    if (metrics_rating == "tbdg"):
        recomendations = get_recommendation_tbdg_item(bookid, number_of_book, metric_similar)
        return recomendations
    elif (metrics_rating == "tbdgcts"):
        recomendations = get_recommendation_tbdgcts_item(bookid, number_of_book, metric_similar)
        return recomendations
    elif (metrics_rating == "kdg"):
        recomendations = get_recommendation_kdg_item(bookid, number_of_book, metric_similar)
        return recomendations

In [ ]:
# Thử nghiệm
book =  58543601.0
number_of_book = 10
metrics_rating = "kdg" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "cosine" # 1 trong 2 ["cosine", "peason"]
book_recommend = get_recommendation_item(book, number_of_book, metrics_rating, metrics_similar)

In [ ]:
book_recommend 

[(26044245.0, 5.120204603580563),
 (21474832.0, 5.120204603580563),
 (21151479.0, 5.120204603580563),
 (12205402.0, 5.120204603580563),
 (11790919.0, 5.120204603580563),
 (8094853.0, 5.120204603580563),
 (7178520.0, 5.120204603580563),
 (7107873.0, 5.120204603580563),
 (6087484.0, 5.120204603580563),
 (5770824.0, 5.120204603580563),
 (5359550.0, 5.120204603580563),
 (1997514.0, 5.120204603580563),
 (708667.0, 5.120204603580563),
 (16754605.0, 4.992753623188406),
 (10011149.0, 4.992753623188406),
 (6158890.0, 4.992753623188406),
 (456030.0, 4.992753623188406),
 (632189.0, 4.985966292871664),
 (11634258.0, 4.9689440993788825),
 (11525835.0, 4.9689440993788825),
 (5805473.0, 4.9689440993788825),
 (522878.0, 4.9689440993788825),
 (385406.0, 4.9689440993788825),
 (311452.0, 4.9689440993788825),
 (33282.0, 4.9689440993788825),
 (25960212.0, 4.9480381760339345),
 (25058444.0, 4.9480381760339345),
 (21600295.0, 4.9480381760339345),
 (21251784.0, 4.9480381760339345),
 (20299308.0, 4.94803817603

# Đánh giá phương pháp

In [ ]:
def get_recommendation_4test_item(X_test, number_of_book, metrics_rating, metric_similar):
    X_test_pred = []
    X_test_userid = X_test['user_id'].tolist()
    X_test_bookId = X_test['product_id'].tolist()
    print("Dự đoán cho tập test với {} điểm dữ liệu".format(len(X_test_bookId)))
    for i in range(len(X_test_bookId)):
        list_R = get_recommendation_item(X_test_bookId[i], number_of_book, metrics_rating, metrics_similar)
        check = 0
        for j in list_R:
            if(X_test_userid[i] == j[0]):
                X_test_pred.append(j[1])
                check = 1
        if(check == 0 ):
            X_test_pred.append(0)
        print('----------dự đoán cho dòng thứ {}'.format(i), 'là {}'.format(X_test_pred[i]))
    return X_test_pred

## Dự đoán và đánh giá bằng pp KDG và cosine

In [ ]:
import time
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
import time
start = time.time()
user = "U1"
number_similar = 100
metrics_rating = "kdg" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "cosine" # 1 trong 2 ["cosine", "peason"]
X_test_pred_rating = get_recommendation_4test_item(test, number_similar, metrics_rating, metrics_similar)

end = time.time()

print("Tổng thời gian dự đoán :", end - start)

Dự đoán cho tập test với 1423 điểm dữ liệu
----------dự đoán cho dòng thứ 0 là 4.8335183661507175
----------dự đoán cho dòng thứ 1 là 2.9677633765370555
----------dự đoán cho dòng thứ 2 là 4.8533036352316845


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


----------dự đoán cho dòng thứ 3 là 5.1237374629655905
----------dự đoán cho dòng thứ 4 là 0
----------dự đoán cho dòng thứ 5 là 4.684684684684685
----------dự đoán cho dòng thứ 6 là 4.786383900767331
----------dự đoán cho dòng thứ 7 là 0
----------dự đoán cho dòng thứ 8 là 5.0653307876386595
----------dự đoán cho dòng thứ 9 là 0
----------dự đoán cho dòng thứ 10 là 0
----------dự đoán cho dòng thứ 11 là 5.116376531270148
----------dự đoán cho dòng thứ 12 là 4.987037766754748
----------dự đoán cho dòng thứ 13 là 4.81599992339412
----------dự đoán cho dòng thứ 14 là 0
----------dự đoán cho dòng thứ 15 là 4.716192202904452
----------dự đoán cho dòng thứ 16 là 0
----------dự đoán cho dòng thứ 17 là 4.636487523900116
----------dự đoán cho dòng thứ 18 là 0
----------dự đoán cho dòng thứ 19 là 5.085607089154586
----------dự đoán cho dòng thứ 20 là 0
----------dự đoán cho dòng thứ 21 là 4.987438619614444
----------dự đoán cho dòng thứ 22 là 4.576712962962963
----------dự đoán cho dòng thứ 23 

In [ ]:
import csv
X_test_pred_rating1= pd.DataFrame(data=X_test_pred_rating)
#X_test_pred_rating1 = X_test_pred_rating
X_test_pred_rating1.to_csv('/content/drive/MyDrive/recomment_system/Do_an/Collaborative_Filtering/Pre-KDG-Cosine.csv')


In [ ]:
X_test_pred_rating = pd.read_csv('/content/drive/MyDrive/recomment_system/Do_an/Collaborative_Filtering/CF_Item_Based/Pre-KDG-Cosine.csv')

In [ ]:
X_test_pred_rating

,0
0,4.833518
1,2.967763
2,4.853304
3,5.123737
4,0.000000
...,...
1392,3.549695
1393,5.022036
1394,5.004676
1395,0.000000


In [ ]:
X_test_pred_rating = list(np.where(np.isnan(X_test_pred_rating), 0, X_test_pred_rating))

X_test_pred_rating_select = []
X_test_true_rating_select = []

for i in range(len(X_test_pred_rating)):
    if (X_test_pred_rating[i] != 0):
        X_test_pred_rating_select.append(X_test_pred_rating[i])
        X_test_true_rating_select.append(test['rating'].tolist()[i])
        X_test_pred_rating_select_round = [np.round(num) for num in X_test_pred_rating_select]

mse = mean_squared_error(X_test_true_rating_select, X_test_pred_rating_select)
rmse = math.sqrt(mse)
mae = mean_absolute_error(X_test_true_rating_select, X_test_pred_rating_select)

X_test_true_rating_select.sort(reverse = True)
max = X_test_true_rating_select[0]
min = X_test_true_rating_select[len(X_test_true_rating_select)-1]
nmae = mae/(max-min)
print('---------------------------------')
print('Đánh giá trên độ đo MSE:', mse)
print('Đánh giá trên độ đo RMSE:', rmse)
print('Đánh giá trên độ đo MAE:', mae)
print('Đánh giá trên độ đo NMAE:', nmae)

---------------------------------
Đánh giá trên độ đo MSE: 0.34464504492245523
Đánh giá trên độ đo RMSE: 0.5870647706364734
Đánh giá trên độ đo MAE: 0.29287279592526017
Đánh giá trên độ đo NMAE: 0.07321819898131504
